# Aerial-Cactus-Identification
Determine whether an image contains a columnar cactus

### Load and preprocessing data

In [4]:
import pandas as pd

In [7]:
ls aerial-cactus-identification_data/

sample_submission.csv* train/
test/                  train.csv*


In [10]:
df = pd.read_csv("./aerial-cactus-identification_data/train.csv")
df.head()

,id,has_cactus
0,0004be2cfeaba1c0361d39e2b000257b.jpg,1
1,000c8a36845c0208e833c79c1bffedd1.jpg,1
2,000d1e9a533f62e55c289303b072733d.jpg,1
3,0011485b40695e9138e92d0b3fb55128.jpg,1
4,0014d7a11e90b62848904c1418fc8cf2.jpg,1
